In [161]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.distance import distance
from sklearn.impute import KNNImputer

In [162]:
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

def visualize_vessel_movements(df):
    """
    Visualize vessel movements on a map with lines and markers for each data point.

    Parameters:
    - df (pandas.DataFrame): A DataFrame with columns ['time', 'latitude', 'longitude', 'vesselId'].

    Returns:
    - A Plotly interactive figure.
    """
    # Ensure 'time' is in datetime format for better tooltip handling
    df['time'] = pd.to_datetime(df['time'])
    
    # Sorting the DataFrame by time to ensure lines are drawn correctly
    df = df.sort_values(by=['vesselId', 'time'])

    # Define a color palette
    color_map = px.colors.qualitative.Plotly

    # Mapping each vessel ID to a color
    unique_vessels = df['vesselId'].unique()
    colors = {vessel_id: color_map[i % len(color_map)] for i, vessel_id in enumerate(unique_vessels)}

    # Create the base map with lines
    fig = px.line_geo(df,
                      lat='latitude',
                      lon='longitude',
                      color='vesselId',
                      color_discrete_map=colors,
                      hover_name='vesselId',
                      hover_data={'time': True, 'latitude': ':.3f', 'longitude': ':.3f'},
                      projection='natural earth',
                      title='Vessel Movements Over Time')

    # Add markers for each data point
    for vessel_id in unique_vessels:
        vessel_data = df[df['vesselId'] == vessel_id]
        fig.add_trace(go.Scattergeo(
            lon=vessel_data['longitude'],
            lat=vessel_data['latitude'],
            mode='markers',
            marker=dict(
                size=8,
                color=colors[vessel_id],
                opacity=0.8,
                line=dict(width=1, color='DarkSlateGrey')
            ),
            name=f'Markers for {vessel_id}',
            hoverinfo='text',
            text=vessel_data.apply(lambda row: f'ID: {vessel_id}<br>Time: {row["time"]}<br>Lat: {row["latitude"]:.3f}<br>Lon: {row["longitude"]:.3f}', axis=1)
        ))

    # Enhancing map and layout details
    fig.update_geos(fitbounds="locations", showcountries=True, countrycolor="RebeccaPurple")
    fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0},
                      coloraxis_colorbar=dict(title="Vessel ID"),
                      title_font_size=20)
    
    return fig

In [483]:
def delta(boat: pd.DataFrame) -> pd.DataFrame:
    """
    Given boat, return dataframe with delta_lat and delta_lon added
    """
    boat.sort_values("time", inplace=True)
    boat.reset_index(drop=True, inplace=True)

    boat["delta_time"] = boat["time"] - boat["time"].shift(1)
    boat["delta_time"] = boat["delta_time"].dt.total_seconds()
    boat["time_cum"] = boat["delta_time"].cumsum()

    boat["delta_lat"] = boat["latitude"] - boat["latitude"].shift(1)
    boat["delta_lon"] = boat["longitude"] - boat["longitude"].shift(1)

    return boat

def speed(boat: pd.DataFrame, upper=0, lower=60) -> pd.DataFrame:
    """
    Given boat, upper and lower speed limit, calculate speed between observations and return boat dataframe with speed.
    Will also fix insane speed using upper/lower. Generally, vessels we care about will not go faster than 60km/h.
    Run AFTER delta()
    """
    boat.sort_values("time", inplace=True)
    boat.reset_index(drop=True, inplace=True)

    boat["speed_from_prev"] = np.nan
    boat["dist_from_prev"] = np.nan
    l = len(boat)

    for i in range(1, l):
        boat.at[i, "dist_from_prev"] = distance(
            (boat.at[i-1, "latitude"], boat.at[i-1, "longitude"]),
            (boat.at[i, "latitude"], boat.at[i, "longitude"])
        ).km

    boat["speed_from_prev"] = (boat["dist_from_prev"] / (boat["delta_time"] / 3600))
    boat["speed_from_prev"].clip(lower=0, upper=60, inplace=True)

    return boat

def moored(boat: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate when we are at a port and then calculate the time at sea, set as seconds.
    """
    boat["at_port"] = boat["navstat"].apply(lambda stat: True if stat == 5 else False)
    indices = boat[boat["at_port"] == True].index

    boat["timestamp_last_port"] = np.nan

    for i in indices:
        boat.at[i, "timestamp_last_port"] = boat.at[i, "time"]

    boat["timestamp_last_port"] = boat["timestamp_last_port"].ffill()

    origin = boat.at[0, "time"]
    boat["timestamp_last_port"] = boat["timestamp_last_port"].fillna(origin)

    #Insh'allah
    boat["time_at_sea"] = boat["time"]-boat["timestamp_last_port"]
    boat["time_at_sea"] = boat["time_at_sea"].apply(lambda x: x if x >= pd.Timedelta(0) else pd.Timedelta(0))
    boat["time_at_sea"] = boat["time_at_sea"].dt.total_seconds()
    boat = boat.drop(columns=["timestamp_last_port"])

    return boat

def navstat(boat: pd.DataFrame, speed_lim = 0.5) -> pd.DataFrame:
    """
    Clip NAVSTAT according to speed_lim. Run AFTER speed()
    """
    for i, row in boat.iterrows():
        speed = row["speed_from_prev"]
        navstat = row["navstat"]

        if navstat > 8:
            if speed <= speed_lim: #Standing still basically
                boat.at[i, "navstat"] = 1 #Anchored
            else:
                boat.at[i, "navstat"] = 0 #Moving with engine

    return boat

def resample(boat: pd.DataFrame) -> pd.DataFrame:
    """
    Resample the boats, kinda stupid implementation but it kinda works???
    """
    t_0 = boat["time"].iloc[0].floor("20min") #Round down to nearest 15-min
    boat.set_index("time", drop=True, inplace=True)

    boat = boat.resample("20min", origin=t_0,).nearest()

    boat.reset_index(drop=False, inplace=True)

    return boat

def cleanUp(data: pd.DataFrame, n=688, resample=False) -> pd.DataFrame:
    wallahi = []

    #Time first
    data["time"] = pd.to_datetime(data["time"])

    #Individual boat stuff
    ids = data["vesselId"].unique()

    for id in ids[:n]:
        boat = data[data["vesselId"] == id].reset_index(drop=True)

        if resample:
            boat = resample(boat)    

        boat = navstat(speed(delta(moored(boat))))

        for _, row in boat.iterrows():
            wallahi.append(row.to_dict())

    wallahi = pd.DataFrame(wallahi)

    #Degree stuff
    wallahi["cog"] = np.sin((wallahi["cog"] * np.pi )/180)
    wallahi["heading"] = np.sin((data["heading"] * np.pi)/180)

    #Fix etaRaw: Set to seconds from 2024-01-01 00:00:00 if possible, if invalid set to pd.NaT and then interpolat
    #using nearest()
    origin = pd.to_datetime("2024-01-01 00:00:00")
    
    wallahi["etaRaw"] = wallahi["etaRaw"].apply(lambda e: "2024-" + e)
    wallahi["etaRaw"] = pd.to_datetime(wallahi["etaRaw"], errors="coerce")
    wallahi["etaRaw"] = wallahi["etaRaw"].apply(lambda e: (e-origin) if not pd.isna(e) else e)
    wallahi["etaRaw"] = wallahi["etaRaw"].dt.total_seconds()
    wallahi["etaRaw"].interpolate("nearest", inplace=True)

    wallahi.bfill(inplace=True)

    wallahi.sort_values("time", inplace=True)

    return wallahi




In [484]:
data = pd.read_csv("../CSV/big_files/ais_train.csv", sep="|")
bruh = cleanUp(data, n=10)

""" boat = data[data["vesselId"] == "61e9f3a8b937134a3c4bfdf7"]
boat["time"] = pd.to_datetime(boat["time"])
boat = speed(delta(boat))

boat["at_port"] = boat["navstat"].apply(lambda stat: True if stat == 5 else False)
indices = boat[boat["at_port"] == True].index

boat["timestamp_last_port"] = np.nan

for i in indices:
    boat.at[i, "timestamp_last_port"] = boat.at[i, "time"]

boat["timestamp_last_port"] = boat["timestamp_last_port"].ffill()

origin = boat.at[0, "time"]
boat["timestamp_last_port"] = boat["timestamp_last_port"].fillna(origin)

#Insh'allah
boat["time_at_sea"] = boat["time"]-boat["timestamp_last_port"]
boat["time_at_sea"] = boat["time_at_sea"].apply(lambda x: x if x >= pd.Timedelta(0) else pd.Timedelta(0))
boat = boat.drop(columns=["timestamp_last_port"])

boat.loc[173:200].head() """

bruh.head()

,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,at_port,time_at_sea,delta_time,time_cum,delta_lat,delta_lon,speed_from_prev,dist_from_prev
0,2024-01-01 00:00:25,-0.970296,0.7,0,0.999391,0,774000.0,-34.74370,-57.85130,61e9f3a8b937134a3c4bfdf7,61d371c43aeaecc07011a37f,False,0.0,22123.0,22123.0,-0.42417,1.07920,17.774078,109.226649
1550,2024-01-01 00:00:36,0.942057,0.0,-6,-0.358368,1,31435200.0,8.89440,-79.47939,61e9f3d4b937134a3c4bff1f,634c4de270937fc01c3a7689,False,0.0,1980.0,1980.0,0.00006,0.00001,0.012230,0.006727
3244,2024-01-01 00:01:45,0.933580,11.0,0,0.573576,0,118800.0,39.19065,-76.47567,61e9f436b937134a3c4c0131,61d3847bb7b7526e1adf3d19,False,0.0,1372.0,1372.0,-0.02408,0.07965,19.373303,7.383381
5080,2024-01-01 00:03:11,0.993768,0.0,0,0.241922,1,31608000.0,-34.41189,151.02067,61e9f3b4b937134a3c4bfe77,61d36f770a1807568ff9a126,False,0.0,3603.0,3603.0,-0.00017,-0.00062,0.059989,0.060039
6386,2024-01-01 00:03:51,-0.559193,19.7,0,0.999391,0,2116800.0,35.88379,-5.91636,61e9f41bb937134a3c4c0087,634c4de270937fc01c3a74f3,False,0.0,85735.0,85735.0,-3.92030,-8.02234,36.086285,859.404909


In [381]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

def evaluate(model: str, y_pred):

    n = len(y_test)
    p = X_test.shape[1]

    mse_latitude = mean_squared_error(y_test[:, 0], y_pred[:, 0])
    mse_longitude = mean_squared_error(y_test[:, 1], y_pred[:, 1])
    r2_latitude = r2_score(y_test[:, 0], y_pred[:, 0])
    r2_latitude_adj = 1 - (1 - r2_latitude) * ((n - 1) / (n - p - 1))
    r2_longitude = r2_score(y_test[:, 1], y_pred[:, 1])
    r2_longitude_adj = 1 - (1 - r2_longitude) * ((n - 1) / (n - p - 1))


    print(f"---- {model} Metrics ----")
    print(f"Mean Squared Error (Latitude): {mse_latitude:.4f}")
    print(f"R-squared (Latitude): {r2_latitude:.4f}, Adjusted R-squared (Latitude): {r2_latitude_adj: .4f}")
    print(f"Mean Squared Error (Longitude): {mse_longitude:.4f}")
    print(f"R-squared (Longitude): {r2_longitude:.4f}, Adjusted R-squared (Latitude): {r2_longitude_adj: .4f}")


In [500]:
X = bruh[["speed_from_prev", "dist_from_prev", "cog", "heading", "navstat", "time_at_sea", "vesselId", "delta_time", "latitude", "longitude", "time"]]
y = bruh[["delta_lat", "delta_lon"]]

X = pd.get_dummies(X, columns=["vesselId"], drop_first=False)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
y_test = y_test.to_numpy()

vessel = "61e9f3a8b937134a3c4bfdf7"

boat_train = X_train[X_train["vesselId_" + vessel]]
boat_train = boat_train[["time", "latitude", "longitude"]]
boat_train["vesselId"] = vessel

X_train.drop(columns=["time", "latitude", "longitude"], inplace=True)
X_test.drop(columns=["time", "latitude", "longitude"], inplace=True)

boat_train.tail()


,time,latitude,longitude,vesselId
1349,2024-04-15 11:16:53,47.54600,-122.52408,61e9f3a8b937134a3c4bfdf7
1350,2024-04-15 11:37:48,47.54590,-122.52378,61e9f3a8b937134a3c4bfdf7
1351,2024-04-15 11:55:45,47.54586,-122.52383,61e9f3a8b937134a3c4bfdf7
1352,2024-04-15 12:16:45,47.54587,-122.52381,61e9f3a8b937134a3c4bfdf7
1353,2024-04-15 12:37:45,47.54583,-122.52376,61e9f3a8b937134a3c4bfdf7


In [501]:
# --- XGBOOST ---
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

evaluate("XGBoost", y_pred)

X_pred = X_test.copy()
# X_pred["delta_lat"] = pd.DataFrame(y_pred[:,0])

y_pred = pd.DataFrame(y_pred)
y_pred.set_index(X_pred.index, inplace=True)
X_pred["delta_lat"] = y_pred[0]
X_pred["delta_lon"] = y_pred[1]

#Reassembly
boat = X_pred[X_pred["vesselId_" + vessel] == True]
boat = boat[["delta_lat", "delta_lon"]]
boat["vesselId"] = vessel
boat["lat_cum"] = boat["delta_lat"].cumsum()
boat["lon_cum"] = boat["delta_lon"].cumsum()

boat["last_lat"] = boat_train["latitude"].iloc[-1]
boat["last_lon"] = boat_train["longitude"].iloc[-1]

boat["latitude"] = boat["last_lat"] + boat["lat_cum"]
boat["longitude"] = boat["last_lon"] + boat["lon_cum"]

boat["time"] = pd.date_range(boat_train["time"].iloc[-1], freq="20min", periods=len(boat))


boat.drop(columns=["last_lat", "last_lon", "delta_lat", "delta_lon", "lat_cum", "lon_cum"], inplace=True)

boat.head()


---- XGBoost Metrics ----
Mean Squared Error (Latitude): 4.3698
R-squared (Latitude): -0.2950, Adjusted R-squared (Latitude): -0.3005
Mean Squared Error (Longitude): 59.9055
R-squared (Longitude): -0.1537, Adjusted R-squared (Latitude): -0.1587


,vesselId,latitude,longitude,time
1354,61e9f3a8b937134a3c4bfdf7,47.545518,-122.517782,2024-04-15 12:37:45
1355,61e9f3a8b937134a3c4bfdf7,47.545302,-122.518339,2024-04-15 12:57:45
1356,61e9f3a8b937134a3c4bfdf7,47.544990,-122.512361,2024-04-15 13:17:45
1357,61e9f3a8b937134a3c4bfdf7,47.544714,-122.511003,2024-04-15 13:37:45
1358,61e9f3a8b937134a3c4bfdf7,47.544931,-122.505025,2024-04-15 13:57:45


In [502]:
fig = (
    visualize_vessel_movements(boat_train)
)

fig.show()

fig = (
    visualize_vessel_movements(boat)
)
fig.show()